# Sea Surface Temperature

In [ ]:
import sys
sys.path.append("..")
import scipy as sp
import numpy as np
import xarray as xr
import seaborn as sns
import cmocean
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
from OHC import t2da, t2ds
from SST import SST_index, EOF_SST_analysis
from maps import map_robinson, map_eq_earth
from grid import find_array_idx
from paths import path_results, path_samoc, file_ex_ocn_ctrl, file_ex_ocn_rect, file_ex_ocn_lpd
from regions import boolean_mask, SOM_area, global_ocean, SOM_mask
from plotting import shifted_color_map, discrete_cmap
from timeseries import IterateOutputCESM, lowpass
from xr_DataArrays import xr_AREA
from xr_regression import xr_linear_trends_2D, xr_linear_trend, ocn_field_regression

# global yearly avg. SST map and trends

In [ ]:
SST_yrly_ctrl = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_ctrl.nc')
SST_yrly_rcp  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_rcp.nc' )
SST_yrly_lpd  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_lpd.nc' )
SST_yrly_lpi  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_lpi.nc' )

# SST indices

In [ ]:
MASK_ocn  = boolean_mask('ocn'     , 0)
MASK_low  = boolean_mask('ocn_low' , 0)
MASK_rect = boolean_mask('ocn_rect', 0)

In [ ]:
TAREA       = xr_AREA('ocn')
AREA_low    = xr_AREA('ocn_low')
AREA_rect   = xr_AREA('ocn_rect')

# Southern Ocean

## Southern Ocean Mode index
annual SST anomaly (50S-35S, 0E-50W), here detrended

In [ ]:
SOMi_area     = TAREA.where(SOM_mask('ocn')    ).sum()
SOMi_area_low = AREA_low.where(SOM_mask('ocn_low')).sum()
print(SOMi_area, SOMi_area_low)

In [ ]:
%%time
# 30 sec
dims = ('nlat', 'nlon')
SOMi_ctrl = SST_index(xa_SST=SST_yrly_ctrl, AREA=TAREA, index_loc=None, AREA_index=SOMi_area, MASK=SOM_mask('ocn'), dims=dims)
SOMi_rcp  = SST_index(xa_SST=SST_yrly_rcp , AREA=TAREA, index_loc=None, AREA_index=SOMi_area, MASK=SOM_mask('ocn'), dims=dims)
SOMi_lpd  = SST_index(xa_SST=SST_yrly_lpd , AREA=AREA_low, index_loc=None, AREA_index=SOMi_area_low, MASK=SOM_mask('ocn_low'), dims=dims)
SOMi_lpi  = SST_index(xa_SST=SST_yrly_lpi , AREA=AREA_low, index_loc=None, AREA_index=SOMi_area_low, MASK=SOM_mask('ocn_low'), dims=dims)

In [ ]:
def lin_detrend(ts):
    lp = np.polyfit(ts.time, ts, 1)
    detr = ts - (lp[1] + lp[0]*ts.time)
    return detr

In [ ]:
SOMi_detr_ctrl = lin_detrend(SOMi_ctrl)
SOMi_detr_rcp  = lin_detrend(SOMi_rcp )
SOMi_detr_lpd  = lin_detrend(SOMi_lpd )
SOMi_detr_lpi  = lin_detrend(SOMi_lpi )

SOMi_detr_ctrl.to_netcdf(f'{path_results}/SST/SOM_index_ctrl.nc')
SOMi_detr_rcp .to_netcdf(f'{path_results}/SST/SOM_index_rcp.nc' )
SOMi_detr_lpd .to_netcdf(f'{path_results}/SST/SOM_index_lpd.nc' )
SOMi_detr_lpi .to_netcdf(f'{path_results}/SST/SOM_index_lpi.nc' )

In [ ]:
f,ax = plt.subplots(2,1,figsize=(12,8), sharex=True)

for i in range(2):
    ax[i].tick_params(labelsize=14)
    ax[1].set_xlabel('time [years]', fontsize=16)
    
ax[0].set_ylabel('SOM area temperature [$^\circ$C]', fontsize=16)
ax[0].plot(SOMi_ctrl.time/365+1900, SOMi_ctrl, lw=.5, ls=':', c='C0', label='yearly data')
ax[0].plot(SOMi_rcp .time/365+ 200, SOMi_rcp , lw=.5, ls=':', c='C1')
ax[0].plot(SOMi_lpd .time/365+1400, SOMi_lpd , lw=.5, ls=':', c='C2')
ax[0].plot(SOMi_lpi .time/365-1600, SOMi_lpi , lw=.5, ls=':', c='C3')

ax[0].plot(SOMi_ctrl.time/365+1900, lowpass(SOMi_ctrl.values, 15), c='C0', label='15 year lowpass filter')
ax[0].plot(SOMi_rcp .time/365+ 200, lowpass(SOMi_rcp .values, 15), c='C1')
ax[0].plot(SOMi_lpd .time/365+1400, lowpass(SOMi_lpd .values, 15), c='C2')
ax[0].plot(SOMi_lpi .time/365-1600, lowpass(SOMi_lpi .values, 15), c='C3')

ax[0].legend(fontsize=14, ncol=2, frameon=False)

ax[1].set_ylabel('SOM index [$^\circ$C]', fontsize=16)
ax[1].axhline(0, c='k', lw=.5)
ax[1].plot(SOMi_ctrl.time/365+1900, lowpass(SOMi_detr_ctrl.values,15), c='C0', label='detrended, 15 year lowpass filter')
ax[1].plot(SOMi_rcp .time/365+ 200, lowpass(SOMi_detr_rcp .values,15), c='C1')
ax[1].plot(SOMi_lpd .time/365+1400, lowpass(SOMi_detr_lpd .values,15), c='C2')
ax[1].plot(SOMi_lpi .time/365-1600, lowpass(SOMi_detr_lpi .values,15), c='C3')

ax[1].legend(fontsize=14, frameon=False)

ax[0].text(   0, 12.5, 'pre-ind. low' , fontsize=16, color='C3')
ax[0].text(1550, 12.5, 'pres. day low', fontsize=16, color='C2')
ax[0].text(2000, 12.5, 'CTRL'         , fontsize=16, color='C0')
ax[0].text(2200, 12.5, 'RCP'          , fontsize=16, color='C1')


plt.tight_layout()
plt.savefig(f'{path_results}/SST/SOM_overview')

In [ ]:
SOMi_detr_ctrl = xr.open_dataarray(f'{path_results}/SST/SOM_index_ctrl.nc', decode_times=False)
SOMi_detr_rcp  = xr.open_dataarray(f'{path_results}/SST/SOM_index_rcp.nc' , decode_times=False)


In [ ]:
f = plt.figure(figsize=(8,5))
ax = f.add_axes([0.13,0.13,.85,.85])
plt.tick_params(labelsize=14)

# plt.axhline(0,c='k', lw=.5)
plt.plot(SOMi_ctrl.time/365     , SOMi_ctrl, c='C0', lw=.5)
plt.plot(SOMi_rcp .time/365-1800, SOMi_rcp , c='C1', lw=.5)

plt.plot(SOMi_ctrl.time/365     , SOMi_trend_ctrl[1] + SOMi_trend_ctrl[0]*SOMi_ctrl.time, ls='--')
plt.plot(SOMi_rcp .time/365-1800, SOMi_trend_rcp [1] + SOMi_trend_rcp [0]*SOMi_rcp.time , ls='--')

L1, = plt.plot(SOMi_ctrl.time/365     , SOMi_ctrl.rolling({'time':5}, center=True).mean(), c='C0', lw=2, label='CTRL')
L2, = plt.plot(SOMi_rcp .time/365-1800, SOMi_rcp .rolling({'time':5}, center=True).mean(), c='C1', lw=2, label='RCP')

plt.text(.02,.95, r'[35,50$^\circ$S]$\times$[-50,0$^\circ$E]', ha='left',transform=ax.transAxes, fontsize=16)
plt.xlabel('time [years]', fontsize=16)
plt.ylabel('raw SOM index [K]', fontsize=16)
plt.legend(fontsize=16, ncol=2, loc=4, handles=[L1, L2])
# plt.savefig(f'{path_results}/SST/SOM_raw')

In [ ]:
f = plt.figure(figsize=(8,5))
ax = f.add_axes([0.13,0.13,.85,.85])
plt.tick_params(labelsize=14)

plt.axhline(0,c='k', lw=.5)
plt.plot(SOMi_ctrl.time/365     , SOMi_detr_ctrl, c='C0', lw=.5)
plt.plot(SOMi_rcp .time/365-1800, SOMi_detr_rcp , c='C1', lw=.5)
plt.plot(SOMi_ctrl.time/365     , SOMi_detr_ctrl.rolling({'time':5}, center=True).mean(), c='C0', lw=2, label='CTRL')
plt.plot(SOMi_rcp .time/365-1800, SOMi_detr_rcp .rolling({'time':5}, center=True).mean(), c='C1', lw=2, label='RCP')

plt.text(.02,.95, r'[35,50$^\circ$S]$\times$[-50,0$^\circ$E]', ha='left',transform=ax.transAxes, fontsize=16)
plt.xlabel('time [years]', fontsize=16)
plt.ylabel('SOM index [K]', fontsize=16)
plt.legend(fontsize=16, ncol=2)
# plt.savefig(f'{path_results}/SST/SOM_index')